<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#загрузка-и-анализ-данных" data-toc-modified-id="загрузка-и-анализ-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>загрузка и анализ данных</a></span></li><li><span><a href="#обработка--данных" data-toc-modified-id="обработка--данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>обработка  данных</a></span></li><li><span><a href="#подготовка--данных" data-toc-modified-id="подготовка--данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>подготовка  данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#модель-LogisticRegression" data-toc-modified-id="модель-LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>модель LogisticRegression</a></span></li><li><span><a href="#модель-DecisionTreeClassifier" data-toc-modified-id="модель-DecisionTreeClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>модель DecisionTreeClassifier</a></span></li><li><span><a href="#модель-LGBMClassifier" data-toc-modified-id="модель-LGBMClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>модель LGBMClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

**Описание исследования:**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цель исследования:**

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Задачи исследования:**

- 1) подготовить и обработать данные для обучения моделей
- 2) выбрать и обучить несколько моделей.
- 3) проанализировать результат и выбрать лучшую модель для дальнейшего использования в приложении

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### загрузка и анализ данных

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
import time

from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
stop_words = stopwords.words('english')

tqdm.pandas()

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
#загрузим данные в data
data=pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
# получим общую информацию
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
data.shape

(159292, 3)

In [5]:
#посмотрим на начало и конец таблицы
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.tail()

,Unnamed: 0,text,toxic
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0
159291,159450,"""\nAnd ... I really don't think you understand...",0


In [7]:
# проверим наличие дубликатов
print('Количество дубликатов =', data.duplicated().sum())

Количество дубликатов = 0


In [8]:
#проверим налоичие пропусков
print('Количество пропусков =', data.isna().sum())

Количество пропусков = Unnamed: 0    0
text          0
toxic         0
dtype: int64


In [9]:
#Посмотрим сколько у нас токсичных/нектоксичных текстов
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [10]:
#Выведем соотношение
print('отношение токсичный коментарием ко всем данным =', data['toxic'].value_counts()[1] / data['toxic'].shape)


отношение токсичный коментарием ко всем данным = [0.10161213]


вывод:

- фаил data состоит из 3 столбцов и 159292 строк
- в столбце text размещены комментарии на английском
- в стобце toxic критерии 0 не токсичный коментарий, 1 - токсичный
- столбец Unnamed: 0 - без названия, значения в нем обычные цифры больше похоже на индексное значение, данный фаил нужно удалить, не имеет никакого значения
- в фаилах не обнаружено пропуском и дубликатов
- отношение токсичных коментарием ко всем коментария составляет 10,16%
- в коментария имеются странные значения \n

### обработка  данных

в данном разделе проведем обработку данных, удалим не нужный столбец, очистим и лемматизируем текст.

In [11]:
#удалим столбец Unnamed:
data = data.drop(['Unnamed: 0'], axis=1)

In [12]:
# проверим
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [13]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [14]:
#создадим функцию очищения текстов:
def clear_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    text = ' '.join(text.split())
    return text

# Метод для лемматизации текста
#def lemmatize(text):
   # m = WordNetLemmatizer()
   # txt_list =  nltk.word_tokenize(text)
   # txt_list = ' '.join([m.lemmatize(word) for word in txt_list])
  #  return txt_list

In [15]:
%%time
#очистим:
data['lemm_text'] = data['text'].apply(clear_text) 

CPU times: user 3.94 s, sys: 80.6 ms, total: 4.02 s
Wall time: 4.03 s


In [16]:
#создадим функцию РОS-тэгирования:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,               #прилагательное
                "N": wordnet.NOUN,              #существительное
                "V": wordnet.VERB,              #глагол
                "R": wordnet.ADV                #наречие
               }  
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

#Метод для лемматизации текста:
def lemm_text(text):
    text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(text)]
    return ' '.join(text)

In [18]:
%%time
#леммализирую тексты постов:
data['lemm_text'] = data['lemm_text'].progress_apply(lemm_text) 

100%|██████████| 159292/159292 [18:16<00:00, 145.27it/s]

CPU times: user 16min 30s, sys: 1min 29s, total: 18min
Wall time: 18min 16s


In [19]:
#применим методы и добавим новый столбец lemm_text с 
#data['lemm_text'] = data['text'].progress_apply(lambda x: lemmatize(clear_text(x)))

In [22]:
data.head()

,text,toxic,lemm_text
0,explanation why the edits made under my userna...,0,explanation why the edits make under my userna...
1,d'aww he matches this background colour i'm se...,0,d'aww he match this background colour i 'm see...
2,hey man i'm really not trying to edit war it's...,0,hey man i 'm really not try to edit war it 's ...
3,more i can't make any real suggestions on impr...,0,more i ca n't make any real suggestion on impr...
4,you sir are my hero any chance you remember wh...,0,you sir be my hero any chance you remember wha...


### подготовка  данных

In [23]:
# делим данные на признаки и целевую переменную 
features = data['lemm_text']
target = data['toxic']

In [24]:
#разделяю выборки на обучающую и тестовую в соотношении 90/10:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=.1, 
                                                                              random_state=12345)

In [25]:
print(features_train.shape)
print(target_train.shape)         
print(features_test.shape)      
print(target_test.shape)           

(143362,)
(143362,)
(15930,)
(15930,)


Векторизуем данные с учетом стоп-слов

In [26]:
stopwords_ = set(stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords_)
tf_idf = count_tf_idf.fit_transform(tqdm(features_train))

100%|██████████| 143362/143362 [00:05<00:00, 27774.87it/s]


In [27]:
tf_idf_test = count_tf_idf.transform(features_test)

In [28]:
print("Размер матрицы обучающей выборки:", tf_idf.shape)
print("Размер матрицы тестовой выборки:", tf_idf_test.shape)

Размер матрицы обучающей выборки: (143362, 142866)
Размер матрицы тестовой выборки: (15930, 142866)


### Вывод

данном разделе провели: загрузку данных, их обработку и подготовку для обучения.

в разделе 1.1:
- фаил data состоит из 3 столбцов и 159292 строк
- в столбце text размещены комментарии на английском
- в стобце toxic критерии 0 не токсичный коментарий, 1 - токсичный
- столбец Unnamed: 0 - без названия, значения в нем обычные цифры больше похоже на индексное значение, данный фаил нужно удалить, не имеет никакого значения
- в фаилах не обнаружено пропуском и дубликатов
- отношение токсичных коментарием ко всем коментария составляет 10,16%
- в коментария имеются странные значения \n

в разделе 1.2:
- удалили лишний столбец Unnamed: 0
- провели очистку текста
- провели лиматизацию

в разделе 1.3:
- провели подготовку данных для обучения
- разделили выборку обучающую и тестовую (90 на 10)
- обучающую выборку преобразовали

приступим к обучению моделей

## Обучение

в данной разделе проведем обучернию трех моделей: LogisticRegression, DecisionTreeClassifier, LGBMClassifier

### модель LogisticRegression

Обучим логистическую модель, 

In [38]:
%%time
model_lr = LogisticRegression(fit_intercept=True, 
                              class_weight='balanced', 
                              random_state=12345,
                              solver='liblinear'
                             )
#Определяю словарь с набором параметров
parametrs = {'C': [0.1, 1, 5]}


model_lr_grid = GridSearchCV(model_lr, parametrs, scoring='f1', cv=3)
model_lr_grid.fit(tf_idf, target_train)

f1_lr = (model_lr_grid.best_score_.mean()).round(2)
print('f1 модели Логистической регрессии: = ', f1_lr)

f1 модели Логистической регрессии: =  0.75
CPU times: user 1min 14s, sys: 1min 16s, total: 2min 30s
Wall time: 2min 30s


### модель DecisionTreeClassifier

In [33]:
%%time
model_dtc = DecisionTreeClassifier()
parameters = [{'max_depth':[x for x in range(75,151,25)],
                'random_state':[12345],
                'class_weight':['balanced']}]

gscv_dtc = GridSearchCV(model_dtc, parameters, scoring='f1', cv=3, n_jobs=-1)

gscv_dtc.fit(tf_idf,target_train)


f1_dtc = (gscv_dtc.best_score_.mean()).round(2)
#mts = gscv_dtc.cv_results_['mean_test_score']
#f1_dtc = (max(mts)).round(2)

print('F1 дерева решений =', f1_dtc)
print('при параметрах', gscv_dtc.best_params_)

F1 дерева решений = 0.65
при параметрах {'class_weight': 'balanced', 'max_depth': 150, 'random_state': 12345}
CPU times: user 15min 40s, sys: 1.79 s, total: 15min 41s
Wall time: 15min 43s


### модель LGBMClassifier

In [31]:
%%time
lgbm_modelc = LGBMClassifier(random_state = 12345, learning_rate = 0.5, max_depth = 5, num_leaves = 10)
scores = cross_val_score(lgbm_modelc, tf_idf, target_train, cv=3, scoring='f1')
f1_lgbmc =  scores.mean().round(2)
print('Взвешенная метрика F1 по результатам кросс-валидации, LGBM:', f1_lgbmc)

Взвешенная метрика F1 по результатам кросс-валидации, LGBM: 0.74
CPU times: user 7min 50s, sys: 1.57 s, total: 7min 51s
Wall time: 7min 54s


## Выводы

построим таблицу моделей и результатов обучения f1

In [34]:
models = [['LogisticRegression', f1_lr], 
          ['DecisionTreeClassifier', f1_dtc], 
          ['LGBMClassifier', f1_lgbmc], 
         ]
          
data_models = pd.DataFrame(models, columns = ['модель', 'f1']) 
          
print(data_models)

                   модель    f1
0      LogisticRegression  0.75
1  DecisionTreeClassifier  0.65
2          LGBMClassifier  0.74


проверим LogisticRegression на тестовой выборке

In [36]:
predictions = model_lr_grid.predict(tf_idf_test)
f1_lr_test = (f1_score(target_test, predictions)).round(2)

print('F1 модели LogisticRegression на тестовой выборке:=', f1_lr_test)

F1 модели LogisticRegression на тестовой выборке:= 0.76


лучшей моделью обучения текстов показала себя LogisticRegression с метриков f1 = 0.75

В данной работе целью исследования было обучить модель классифицировать комментарии на позитивные и негативные. 

В вашем распоряжении набор данных с разметкой о токсичности правок. Данный фаил data состоит из 3 столбцов и 159292 строк.

в первой часте работы был проведен анализ данных, проверка на наличие пропусков, дубликатов, в каком формате находятся данные, есть ошибки и некорректные данные

отношение токсичных коментариев ко всем коментариям составляет 10,16%

провели коррекцию данных
- удалили лишний столбец Unnamed: 0
- провели очистку текста
- провели лиматизацию

для обчениея моделей провели подготовку данных для обучения:

- разделили выборку обучающую и тестовую (90 на 10)
- обучающую выборку преобразовали

во 2 разделе провели обучерние трех моделей: LogisticRegression, DecisionTreeClassifier, LGBMClassifier с подбором лучших парамертов

В данной работе были обучены модели для того чтобы классифицировать комментарии на позитивные и негативные.

Необходимо было построить модель со значением метрики качества F1 не меньше 0.75.

с данной работой справилась только одна модель LogisticRegression, все остальные были ниже 0.75
